In [2]:
from azureml.core import Workspace, Experiment, Run
from azureml.core.authentication import ServicePrincipalAuthentication



#ws = Workspace.from_config()
spa = ServicePrincipalAuthentication("9d396536-b01d-424e-ace8-2a339bc8e502", "6df41538-cedc-4dcc-b67a-4b5f767b5013","pG68v79DeZ]Qdz@EmGRjWmVzw-FaYit?" , _enable_caching=False)
ws = Workspace.get(name='acc_ws',auth=spa, subscription_id='f30951ea-6926-40a7-9c19-adeba1c67ec4', resource_group='acc_ws_rg')
exp = Experiment(workspace=ws, name="exp")

### CREATING A COMPUTE TARGET

#### CPU CLUSTER

In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster2"

# Verify that cluster does not exist already
try:
    comp = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    comp = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

comp.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


#### VIRTUAL MACHINE

In [3]:
from azureml.core.compute import RemoteCompute, ComputeTarget

# Create the compute config 
compute_target_name = "attach-dsvm"
attach_config = RemoteCompute.attach_configuration(address = "52.166.130.102",
                                                 
                                                

# If you authenticate with SSH keys instead, use this code:
                                                  ssh_port=22,
                                                  username='accdsvm',
                                                  password=None,
                                                  private_key_file="C:/Users/ygulf/.ssh/id_rsa")

# Attach the compute
comp = ComputeTarget.attach(ws, compute_target_name, attach_config)

comp.wait_for_completion(show_output=True)

SucceededProvisioning operation finished, operation "Succeeded"


### RUN SCRIPT WITH ESTIMATOR

In [14]:
from azureml.train.estimator import Estimator

script_params = {
    '--account_name': "lakestorage"
    
}


sk_est = Estimator(source_directory='C:/Users/ygulf/Desktop/Intertech',
                   compute_target=comp,script_params=script_params,
                   entry_script='train_churn_params.py',conda_packages=['scikit-learn'], pip_packages=['azureml-sdk','azureml-core','azure'])

In [15]:
run = exp.submit(sk_est)
run.wait_for_completion(show_output = True)

WARNING - Validation identified the following error(s): 
C:/Users/ygulf/Desktop/Intertech\train_churn_params.py:19:10: F821 undefined name 'argparse'


RunId: exp_1575315306_fe3d6f02
Web View: https://ml.azure.com/experiments/exp/runs/exp_1575315306_fe3d6f02?wsid=/subscriptions/f30951ea-6926-40a7-9c19-adeba1c67ec4/resourcegroups/acc_ws_rg/workspaces/acc_ws

Streaming azureml-logs/20_image_build_log.txt

2019/12/02 19:35:17 Downloading source code...
2019/12/02 19:35:18 Finished downloading source code
2019/12/02 19:35:18 Creating Docker network: acb_default_network, driver: 'bridge'
2019/12/02 19:35:19 Successfully set up Docker network: acb_default_network
2019/12/02 19:35:19 Setting up Docker configuration...
2019/12/02 19:35:20 Successfully set up Docker configuration
2019/12/02 19:35:20 Logging in to registry: accws2463cd3b.azurecr.io
2019/12/02 19:35:21 Successfully logged into accws2463cd3b.azurecr.io
2019/12/02 19:35:21 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2019/12/02 19:35:21 Scanning for dependencies...
2019/12/02 19:35:22 Successfully scanned dependencies
201


xz-5.2.4             | 366 KB    |            |   0% 
xz-5.2.4             | 366 KB    | #########4 |  95% 
xz-5.2.4             | 366 KB    | ########## | 100% 

readline-6.2         | 713 KB    |            |   0% 
readline-6.2         | 713 KB    | ########3  |  83% 
readline-6.2         | 713 KB    | ########## | 100% 

libstdcxx-ng-9.2.0   | 4.5 MB    |            |   0% 
libstdcxx-ng-9.2.0   | 4.5 MB    | #######6   |  76% 
libstdcxx-ng-9.2.0   | 4.5 MB    | #########9 |  99% 
libstdcxx-ng-9.2.0   | 4.5 MB    | ########## | 100% 

libcblas-3.8.0       | 10 KB     |            |   0% 
libcblas-3.8.0       | 10 KB     | ########## | 100% 

zlib-1.2.11          | 105 KB    |            |   0% 
zlib-1.2.11          | 105 KB    | ########## | 100% 

libblas-3.8.0        | 10 KB     |            |   0% 
libblas-3.8.0        | 10 KB     | ########## | 100% 

tk-8.5.19            | 1.9 MB    |            |   0% 
tk-8.5.19            | 1.9 MB    | #######7   |  78% 
tk-8.5.19            

  Created wheel for json-logging-py: filename=json_logging_py-0.2-cp36-none-any.whl size=3924 sha256=6d98d890f184d1e6de931342de6cedef94351157ae4ca30e783474b3f2b2b674
  Stored in directory: /root/.cache/pip/wheels/0d/2e/1c/c638b7589610d8b9358a6e5eb008edacb8b3e9b6d1edc9479f
  Created wheel for pathspec: filename=pathspec-0.6.0-cp36-none-any.whl size=26671 sha256=c4af823c3168d59440c0867051ca54dce1f8930e554fcdf909ef2c43a4517026
  Stored in directory: /root/.cache/pip/wheels/62/b8/e1/e2719465b5947c40cd85d613d6cb33449b86a1ca5a6c574269
  Created wheel for liac-arff: filename=liac_arff-2.4.0-cp36-none-any.whl size=13333 sha256=00f49273ab8ef647ca681196df22a728bc44d112aca4a3870818ecae66d51b79
  Stored in directory: /root/.cache/pip/wheels/d1/6a/e7/529dc54d76ecede4346164a09ae3168df358945612710f5203
  Created wheel for dill: filename=dill-0.3.1.1-cp36-none-any.whl size=78532 sha256=e7d5575daf76826a70d2fe9ec49f71c7ea0e9133bdc9bd57336c9fb87f87a9aa
  Stored in directory: /root/.cache/pip/wheels/59/b1

ERROR: azure-mgmt 4.0.0 has requirement azure-mgmt-keyvault~=1.0, but you'll have azure-mgmt-keyvault 2.0.0 which is incompatible.
ERROR: azure-mgmt 4.0.0 has requirement azure-mgmt-resource~=2.0, but you'll have azure-mgmt-resource 6.0.0 which is incompatible.
ERROR: azure-mgmt 4.0.0 has requirement azure-mgmt-storage~=2.0, but you'll have azure-mgmt-storage 6.0.0 which is incompatible.
ERROR: azure 4.0.0 has requirement azure-graphrbac~=0.40.0, but you'll have azure-graphrbac 0.61.1 which is incompatible.
#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_081bdca68335633bc117baec729e35d4
#
# To deactivate an active environment, use:
# > source deactivate
#


Removing intermediate container 9ea80cf991c8
 ---> 3e61a258c516
Step 9/14 : ENV PATH /azureml-envs/azureml_081bdca68335633bc117baec729e35d4/bin:$PATH
 ---> Running in dc852a272724
Removing intermediate container dc852a272724
 ---> a319a8e0fc8b
Step 10/14 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-

5df07235e386: Pushed
c7424e5c5e82: Pushed
e1171d4d60ca: Pushed
fa475e766d0d: Pushed
6ef1a8ae63b7: Pushed
340dc32eb998: Pushed
f2608f66a0e3: Pushed
85389f9ead9e: Pushed
0e259b09e5f4: Pushed
9513cdf4e497: Pushed
ccdb13a20bf2: Pushed
7f083f9454c0: Pushed
df18b66efaa6: Pushed
29f36b5893dc: Pushed
2449bac82ec8: Pushed
latest: digest: sha256:cfe9a2944dcdc0254eb4ce0fef15b813765a5c646dc4c6065a42f5b411caf3cb size: 3883
2019/12/02 19:41:04 Successfully pushed image: accws2463cd3b.azurecr.io/azureml/azureml_e4130db0d4b67304bb7cf022db58c2f2:latest
2019/12/02 19:41:04 Step ID: acb_step_0 marked as successful (elapsed time in seconds: 248.142137)
2019/12/02 19:41:04 Populating digests for step ID: acb_step_0...
2019/12/02 19:41:06 Successfully populated digests for step ID: acb_step_0
2019/12/02 19:41:06 Step ID: acb_step_1 marked as successful (elapsed time in seconds: 94.939079)
2019/12/02 19:41:06 The following dependencies were found:
2019/12/02 19:41:06 
- image:
    registry: accws2463cd3b.azu

WARNING - Retrying (Retry(total=2, connect=2, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001B4463A7948>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond')': /history/v1.0/subscriptions/f30951ea-6926-40a7-9c19-adeba1c67ec4/resourceGroups/acc_ws_rg/providers/Microsoft.MachineLearningServices/workspaces/acc_ws/experiments/exp/runs/exp_1575315306_fe3d6f02/details



Streaming azureml-logs/55_azureml-execution-tvmps_7f2abdfb349fda26c9ecaac3a9b024680143636edc7c392a1b4ce0b55062f177_d.txt

2019-12-02T19:45:37Z Starting output-watcher...
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_e4130db0d4b67304bb7cf022db58c2f2
a1298f4ce990: Pulling fs layer
04a3282d9c4b: Pulling fs layer
9b0d3db6dc03: Pulling fs layer
8269c605f3f1: Pulling fs layer
6504d449e70c: Pulling fs layer
8269c605f3f1: Waiting
4e38f320d0d4: Pulling fs layer
b0a763e8ee03: Pulling fs layer
11917a028ca4: Pulling fs layer
a6c378d11cbf: Pulling fs layer
6cc007ad9140: Pulling fs layer
6c1698a608f3: Pulling fs layer
67b79933e3ba: Pulling fs layer
d7dabbe231a9: Pulling fs layer
e6cddcd39924: Pulling fs layer
3dc4c44da36d: Pulling fs layer
f6e9871a1676: Pulling fs layer
c9cf07c45576: Pulling fs layer
6504d449e70c: Waiting
4e38f320d0d4: Waiting
b0a763e8ee03: Waiting
11917a028ca4: Waiting
a6c378d11cbf: Waiting
6cc007ad9140: Waiting
6c1698a608f3: Waiting
67b79933e3ba: 

{'runId': 'exp_1575315306_fe3d6f02',
 'target': 'cpucluster2',
 'status': 'Completed',
 'startTimeUtc': '2019-12-02T19:45:32.483636Z',
 'endTimeUtc': '2019-12-02T19:48:13.290685Z',
 'properties': {'_azureml.ComputeTargetType': 'batchai',
  'ContentSnapshotId': 'a2f87991-96d6-48c0-b65a-685cd479830d',
  'AzureML.DerivedImageName': 'azureml/azureml_e4130db0d4b67304bb7cf022db58c2f2',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'runDefinition': {'script': 'train_churn_params.py',
  'arguments': ['--account_name', 'lakestorage'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cpucluster2',
  'dataReferences': {},
  'data': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'environment': {'name': 'Experiment exp Environment',
   'version': 'Autosave_2019-12-02T19:35:08Z_d4731532',
   'python': {'interpreterPath': 'python',

### RUN SCRIPT WITH RUNCONFIG

In [3]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# Create a new runconfig object
run_amlcompute = RunConfiguration()

# Use the cpu_cluster you created above. 
run_amlcompute.target = comp

# Enable Docker
run_amlcompute.environment.docker.enabled = True

# Set Docker base image to the default CPU-based image
run_amlcompute.environment.docker.base_image = DEFAULT_CPU_IMAGE

# Use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_amlcompute.environment.python.user_managed_dependencies = False

# Specify CondaDependencies obj, add necessary packages
run_amlcompute.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas'],
                                                                           pip_packages=['azureml-sdk','azureml-core','azure'])

print('Base Docker image is:', run_amlcompute.environment.docker.base_image)


NameError: name 'comp' is not defined

In [13]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory = "C:/Users/ygulf/Desktop/Intertech", script = 'train_churn.py', run_config = run_amlcompute)
run = exp.submit(src)
run.wait_for_completion(show_output = True)

RunId: exp_1575314673_f9f89472
Web View: https://ml.azure.com/experiments/exp/runs/exp_1575314673_f9f89472?wsid=/subscriptions/f30951ea-6926-40a7-9c19-adeba1c67ec4/resourcegroups/acc_ws_rg/workspaces/acc_ws

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt

Streaming azureml-logs/70_driver_log.txt

Starting the daemon thread to refresh tokens in background for process with pid = 8
Entering Run History Context Manager.
/azureml-envs/azureml_66a8db3c761ffdb19b954b48b9cdeae9/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Registering model propensity_model




{'runId': 'exp_1575314673_f9f89472',
 'target': 'attach-dsvm',
 'status': 'Completed',
 'startTimeUtc': '2019-12-02T19:24:42.850932Z',
 'endTimeUtc': '2019-12-02T19:24:56.645685Z',
 'properties': {'_azureml.ComputeTargetType': 'remote',
  'ContentSnapshotId': 'ddc1a72f-2878-4e08-8488-5e1e5ca5fa9b'},
 'inputDatasets': [],
 'runDefinition': {'script': 'train_churn.py',
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'attach-dsvm',
  'dataReferences': {},
  'data': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'environment': {'name': 'Experiment exp Environment',
   'version': 'Autosave_2019-12-02T11:10:02Z_b1f2c696',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': False,
    'condaDependencies': {'channels': ['conda-forge'],
     'dependencies': ['python=3.6.2',
      {'pip': ['azureml-sdk==1.0.76.*', 'azureml-core==1.0.76.*', 'azure']},
      'scikit-learn',
    